In [57]:
import requests
import json
from datetime import datetime, timedelta
import pandas as pd

In [60]:
response_API = requests.get("https://platform.airqo.net/api/v1/devices?tenant=airqo&network=airqo")
print(response_API.status_code)
output_test = json.dumps(response_API.json(),sort_keys=True,indent =4)
device_details = json.loads(output_test)

200


In [61]:
device_details

{'devices': [{'_id': '637e007d7c737c001e799ef3',
   'category': 'lowcost',
   'createdAt': '2022-11-23T11:14:05.820Z',
   'deployment_date': '2022-11-23T11:14:05.814Z',
   'device_codes': ['1951153', '637e007d7c737c001e799ef3', 'aq_g5_115'],
   'device_number': 1951153,
   'height': 0,
   'isActive': False,
   'isPrimaryInLocation': False,
   'long_name': 'Airqo_G5115',
   'maintenance_date': '2022-11-23T11:14:05.814Z',
   'mobility': False,
   'name': 'aq_g5_115',
   'network': 'airqo',
   'nextMaintenance': '2023-02-15T12:57:54.208Z',
   'pictures': [],
   'readKey': 'U2FsdGVkX18wqwh90LX33W8qfE7Ai/Ha46OcjUHM8qsz0FGEPmmMGOemcKQhvRsM',
   'recall_date': '2022-11-23T11:14:05.814Z',
   'status': 'not deployed',
   'visibility': True,
   'writeKey': 'U2FsdGVkX1/oCqkQSIDl1P/4zO2oy1kpcjVsKllzKsDw9VurHeYSGKyZnpNv9AJF'},
  {'_id': '637e004e8022ee001e67f278',
   'category': 'lowcost',
   'createdAt': '2022-11-23T11:13:18.596Z',
   'deployment_date': '2022-11-23T11:13:18.585Z',
   'device_codes

In [89]:
for device in device_details['devices']:
        device_data = []
        status = device.get("isActive",False)
        network = device.get("network", False)
        if network == "airqo":
            if status:
                #print(device['name']) 
                #print("----") 
                device_data.append({"name":device['name']})
                #device_data.append(device['name'])            
        end_date_time = datetime.utcnow()
        start_date_time = end_date_time - timedelta(minutes=60)

        for device in device_data: 
            api_data = []
            url = f'https://platform.airqo.net/api/v1/monitor/devices/uptime?' \
            f'tenant=airqo&' \
            f'startDate={datetime.strftime(start_date_time, "%Y-%m-%dT%H:%M:%S.%fZ")}&' \
            f'endDate={datetime.strftime(end_date_time, "%Y-%m-%dT%H:%M:%S.%fZ")}&' \
            f'device_name={device["name"]}'
            response = requests.get(url).json()
            api_data.extend(response["data"])
            if len(api_data) > 5:
                break
            data_device = []

            for record in api_data:
               for value in record["values"]:
                    data_device.append({"device":value["device_name"],
                                        "time": value["created_at"],
                                        "s1_pm2_5": value["sensor_one_pm2_5"],
                                        "s2_pm2_5": value["sensor_two_pm2_5"]})
            device_data_df = pd.DataFrame(data_device)
            device_data_df.head()
            cleaned_device_data = device_data_df.dropna(subset =["s2","s1","time"])


https://platform.airqo.net/api/v1/monitor/devices/uptime?tenant=airqo&startDate=2023-01-17T21:01:12.388942Z&endDate=2023-01-17T22:01:12.388942Z&device_name=aq_g5_110
https://platform.airqo.net/api/v1/monitor/devices/uptime?tenant=airqo&startDate=2023-01-17T21:01:16.864904Z&endDate=2023-01-17T22:01:16.864904Z&device_name=aq_g5_87
https://platform.airqo.net/api/v1/monitor/devices/uptime?tenant=airqo&startDate=2023-01-17T21:01:20.481788Z&endDate=2023-01-17T22:01:20.481788Z&device_name=aq_g5_84
https://platform.airqo.net/api/v1/monitor/devices/uptime?tenant=airqo&startDate=2023-01-17T21:01:24.144596Z&endDate=2023-01-17T22:01:24.144596Z&device_name=aq_g5_82
https://platform.airqo.net/api/v1/monitor/devices/uptime?tenant=airqo&startDate=2023-01-17T21:01:27.593056Z&endDate=2023-01-17T22:01:27.593056Z&device_name=aq_g5_81
https://platform.airqo.net/api/v1/monitor/devices/uptime?tenant=airqo&startDate=2023-01-17T21:01:31.522007Z&endDate=2023-01-17T22:01:31.522007Z&device_name=aq_g5_80
https://p

In [82]:
def calculate_uptime(): 
    expected_data_count = 60
    actual_count_per_device = cleaned_device_data.grouby(pd.Grouper(key="device")).count()
    actual_count_per_device.reset_index(inplace = True)
    actual_count_per_device["uptime"] = (actual_count_per_device["s2_pm2_5"]/expected_data_count)*100
    
    return actual_count_per_device
    

IndentationError: unexpected indent (2418593992.py, line 3)

In [32]:
# check if the device is active 
# if active, get the device number 
# specify the pollutant 
